In [75]:
import cv2
import numpy as np
from IPython.display import Video 

In [76]:
video_1 = './video_files/intruder_1.mp4'
saved_video = 'saved_video.mp4'
saved_video_ = 'big_saved_video.mp4'
capture_video = cv2.VideoCapture(video_1)
width = int(capture_video.get( cv2.CAP_PROP_FRAME_WIDTH))
hieght = int(capture_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = capture_video.get(cv2.CAP_PROP_FPS)

video_out = cv2.VideoWriter(saved_video, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, hieght) )
video_bigout = cv2.VideoWriter(saved_video_, cv2.VideoWriter_fourcc(*'XVID'), fps, (2*width, 2*hieght) )

In [77]:
bg_sub = cv2.createBackgroundSubtractorKNN(200)

In [78]:
def DrawBanner(frame, text, banner_height_percent=0.08, font_scale =0.8, text_color=(255, 0, 0), font_thickness=2):
    banner_height = int(banner_height_percent * frame.shape[0])
    cv2.rectangle(frame, (0,0), (frame.shape[1], banner_height), (0,0,0), thickness=-1)
    left_offset =20
    location =(left_offset, int(10 + (banner_height_percent * frame.shape[0])/2))
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

In [79]:
kernel = 5 
frame_start = 5
frame_count = 0

while True:
    is_frame, frame = capture_video.read()
    frame_count += 1 
    if is_frame is False:
        break
    else:
        frame_erode_c = frame.copy()
    frame_mask = bg_sub.apply(frame)



    if frame_count > frame_start:
        motion_area = cv2.findNonZero(frame_mask)
        if motion_area is not None: 
            x, y, w, h = cv2.boundingRect(motion_area)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255, 255), 4)
            DrawBanner(frame, 'Intruder Alert')

        frame_mask_erode = cv2.erode(frame_mask, np.ones(kernel, np.uint8))
        motion_area_erode = cv2.findNonZero(frame_mask_erode)
        if motion_area_erode is not None:
            xe, ye, we, he = cv2.boundingRect(motion_area_erode)
            cv2.rectangle(frame_erode_c, (xe, ye), (xe+we, ye+he), (0,255, 255), 4)
            DrawBanner(frame_erode_c, 'Intruder Alert')

        frame_mask_bgr = cv2.cvtColor(frame_mask, cv2.COLOR_GRAY2BGR)
        frame_mask_erode_bgr = cv2.cvtColor(frame_mask_erode, cv2.COLOR_GRAY2BGR)

        contours, hierarchy = cv2.findContours(frame_mask_erode, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            cv2.drawContours(frame_mask_erode_bgr, contours, -1, (255,0,255), 2 )
            conoturs_sorted = sorted(contours, key=cv2.contourArea, reverse=True)

        frame_mask_erode
        video_out.write(frame_erode_c)


capture_video.release()
video_out.release()


